In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')


In [2]:
from crewai import Agent, Task, Crew


In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()


In [4]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool() # search the internet
scrape_tool = ScrapeWebsiteTool() # scrape the internet page
read_resume = FileReadTool(file_path='./fake_resume.md') # read and analyse a file
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md') # RAG over a file


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


In [5]:
from IPython.display import Markdown, display
display(Markdown("./fake_resume.md"))



# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Software Engineering Leader with an 18-year tenure in the technology industry, where he has excelled in leading both remote and in-office engineering teams. His expertise spans across software development, process innovation, and enhancing team collaboration. He is highly proficient in programming languages such as Ruby, Python, JavaScript, TypeScript, and Elixir, alongside deep expertise in various front end frameworks. Noah's significant experience in data science and machine learning has enabled him to spearhead successful deployments of scalable AI solutions and innovative data model development.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Noah has transformed the engineering division into a key revenue pillar for DataKernel, rapidly expanding the customer base from inception to a robust community.
- He spearheaded the integration of cutting-edge AI technologies and the use of scalable vector databases, which significantly enhanced the product's capabilities and market positioning.
- Under his leadership, the team has seen a substantial growth in skill development, with a focus on achieving strategic project goals that have significantly influenced the company's direction.
- Noah also played a critical role in defining the company’s long-term strategic initiatives, particularly in adopting AI technologies that have set new benchmarks within the industry.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed the engineering strategy and operations in close collaboration with C-level executives, playing a pivotal role in shaping the company's technological trajectory.
- Managed diverse teams across multiple time zones in North America and Europe, creating an environment of transparency and mutual respect which enhanced team performance and morale.
- His initiatives in recruiting, mentoring, and retaining top talent have been crucial in fostering a culture of continuous improvement and high performance.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Noah founded InnovPet, a startup focused on innovative IoT solutions for pet care, including a revolutionary GPS tracking collar that significantly enhanced pet safety and owner peace of mind.
- He was responsible for overseeing product development from concept through execution, working closely with engineering teams and marketing partners to ensure a successful market entry.
- Successfully set up an advisory board, established production facilities overseas, and navigated the company through a successful initial funding phase, showcasing his leadership and entrepreneurial acumen.
- Built the initial version of the product leveraging MongoDB

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Noah was instrumental in formulating and executing strategic plans that enhanced inter-departmental coordination and trust, leading to better project outcomes.
- He managed multiple engineering teams, fostering a culture that balances productivity with innovation, and implemented goal-setting frameworks that aligned with the company's long-term goals.
- Was a bery hands on manager using ruby on rails and react to build out a new product.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the formation and development of a high-performance engineering team that was pivotal in increasing company revenue by 500% within two years.
- His leadership in integrating data analytics into business decision-making processes led to the development of a predictive modeling tool that revolutionized customer behavior analysis.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Developed and optimized a central API that significantly improved the functionality used by a large engineering team and thousands of users, enhancing overall system performance and user satisfaction.
- Implemented several critical enhancements, including advanced caching strategies that drastically reduced response times and system loads.

### BetCraft: CTO — 2013 - 2015
- Led the technology strategy post-Series A funding, directly reporting to the board and guiding the company through a phase of significant technological advancement and network expansion.
- His strategic initiatives and partnerships significantly improved platform performance and expanded the company's market reach.
- Helped build his initial product using both React and Angular and got pretty good at it.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is an ideal candidate for senior executive roles, particularly in companies seeking leadership with a robust blend of technical and strategic expertise.


In [6]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application tailoring."
    )
)


In [7]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)


In [8]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)


In [9]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Engineering Interview Preparer",
    goal="Create interview questions and talking points "
         "based on the resume and job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Your role is crucial in anticipating the dynamics of "
        "interviews. With your ability to formulate key questions "
        "and talking points, you prepare candidates for success, "
        "ensuring they can confidently address all aspects of the "
        "job they are applying for."
    )
)


In [10]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences."
    ),
    agent=researcher,
    async_execution=True
)


In [11]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using the GitHub ({github_url}) URLs, and personal write-up "
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)


In [12]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume even but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflrect the candidates "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task], # Depends on the results of the previous 2 tasks, that run in parallel
    agent=resume_strategist
)


In [13]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Create a set of potential interview questions and talking "
        "points based on the tailored resume and job requirements. "
        "Utilize tools to generate relevant questions and discussion "
        "points. Make sure to use these question and talking points to "
        "help the candiadte highlight the main points of the resume "
        "and how it matches the job posting."
    ),
    expected_output=(
        "A document containing key questions and talking points "
        "that the candidate should prepare for the initial interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


In [14]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           profile_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)


In [15]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}


In [16]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)


 [DEBUG]: == Working Agent: Tech Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: == [Tech Job Researcher] Task output: 


 [DEBUG]: == Working Agent: Personal Profiler for Engineers
 [INFO]: == Starting Task: Compile a detailed personal and professional profile using the GitHub (https://github.com/joaomdmoura) URLs, and personal write-up (Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his abili

In [17]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))


# Noah Williams
- Email: noah.williams@example.dev
- Phone: +44 11 111 11111

## Profile
Noah Williams is a distinguished Full Stack Engineer with 18 years of experience spanning front-end and back-end development. He holds an MBA with a strong background in AI and data science, making him a perfect fit for innovative and technically demanding environments. Noah's expertise in multiple programming languages and frameworks, combined with his strategic leadership in deploying scalable solutions, aligns seamlessly with the needs of a Full Stack Engineer at AI Fund.

## Technical Skills
- **Programming Languages:** Proficient in Java, Python, JavaScript, HTML, CSS, and TypeScript.
- **Front-end Development:** Experienced with Angular, React, and jQuery. Capable of crafting appealing visual designs and implementing responsive UIs.
- **Back-end Development:** Skilled in Node.js, Ruby on Rails, and Elixir. Adept at developing well-functioning databases using MySQL and MongoDB, and writing effective APIs.
- **Additional Technologies:** Knowledgeable in web servers like Apache, and UI/UX design principles. Familiar with data privacy regulations and security settings.

## Work History

### DataKernel: Director of Software Engineering (remote) — 2022 - Present
- Transformed the engineering division, integrating cutting-edge AI technologies and scalable databases, enhancing product capabilities and market positioning.
- Led the development and management of a robust software suite, focusing on the full software development life cycle, from conception to deployment.

### DataKernel: Senior Software Engineering Manager (remote) — 2019 - 2022
- Directed engineering strategy and operations, significantly enhancing software responsiveness and efficiency through strategic API implementations and performance optimizations.

### InnovPet: Founder & CEO (remote) — 2019 - 2022
- Developed an innovative IoT solution for pet care, demonstrating strong project management and entrepreneurial skills. Managed full product lifecycle from concept to market entry.

### EliteDevs: Engineering Manager (remote) — 2018 - 2019
- Managed and led multiple engineering teams in the development of new products using React and Ruby on Rails, focusing on both front-end and back-end aspects.

### PrintPack: Engineering Manager (remote) — 2016 - 2018
- Led the development of data-driven tools and strategies, improving business processes and customer interactions through sophisticated data analytics.

### DriveAI: Senior Software Engineer (remote) — 2015 - 2016
- Spearheaded the optimization of central APIs and implemented advanced caching strategies, significantly improving system performance and scalability.

### BetCraft: CTO — 2013 - 2015
- Guided technological strategy and development, enhancing platform capabilities through comprehensive backend solutions and effective frontend implementations.

## Education

### MBA in Information Technology
London Business School - MBA

### Advanced Leadership Techniques
University of London - Certification

### Data Science Specialization
Coursera (Johns Hopkins University) - Certification

### B.Sc. in Computer Science
University of Edinburgh - Bachelor’s degree

Noah Williams is well-prepared to contribute effectively to AI Fund's goals, bringing a wealth of knowledge and experience in full-stack development and strategic project management. His strong communication skills and ability to lead cross-functional teams make him an ideal candidate for the position.

In [18]:
display(Markdown("./interview_materials.md"))


**Document: Key Interview Questions and Talking Points for Noah Williams**

**Introduction to Interview:**
- Begin by thanking Noah for his time and briefly outline the structure of the interview. Ensure to create a comfortable environment that allows for open and detailed responses.

**Technical Skills and Experience:**
1. **Question:** "Can you provide an example of a project where you used both front-end and back-end technologies? How did you ensure the integration was seamless?"
   **Talking Point:** Highlight Noah's experience with Angular, React, Node.js, Ruby on Rails, and Elixir, demonstrating proficiency in full-stack development.

2. **Question:** "Given your extensive background with different databases, could you discuss your approach to developing and managing well-functioning databases at DataKernel?"
   **Talking Point:** Discuss the use of MySQL and MongoDB in past projects, emphasizing his ability to handle data efficiently and securely.

3. **Question:** "In your role at DataKernel, how did you implement effective APIs? Could you walk us through your process from conception to deployment?"
   **Talking Point:** Focus on Noah's strategic and technical capabilities in designing APIs that enhance software responsiveness and efficiency.

**Leadership and Strategic Influence:**
4. **Question:** "As a director, how did you foster a culture of innovation and maintain team motivation during challenging projects?"
   **Talking Point:** Reflect on his leadership roles and the ability to manage cross-functional teams remotely, aligning with the AI Fund's emphasis on team collaboration and leadership.

5. **Question:** "Can you describe a time when you had to troubleshoot, debug, and upgrade a software system? What were the challenges and how did you overcome them?"
   **Talking Point:** This will illustrate Noah's problem-solving skills and his hands-on technical expertise in maintaining and improving software systems.

**Alignment with AI Fund's Goals and Values:**
6. **Question:** "How do you see your experience with AI technologies contributing to our projects at AI Fund?"
   **Talking Point:** Connect his past work with AI and data science to the potential roles and responsibilities at AI Fund, highlighting his readiness to drive innovative solutions.

7. **Question:** "Given our company's commitment to diversity and inclusion, can you share your experience managing diverse teams and fostering an inclusive environment?"
   **Talking Point:** Discuss Noah's experience working with teams across different geographies and cultures, underscoring his fit with the company culture.

**Future Vision and Adaptability:**
8. **Question:** "Where do you see the biggest technological challenges in the next five years, and how would you prepare our team to tackle those challenges?"
   **Talking Point:** This will help gauge Noah's vision for future tech trends and his strategic planning skills.

9. **Question:** "Can you discuss a project where you had to learn a new technology or framework to meet the project's requirements?"
   **Talking Point:** Emphasize his ability to adapt and learn quickly, which is crucial for keeping up with technological advancements.

**Conclusion of Interview:**
- Conclude by asking Noah if he has any questions about the role or the company, ensuring a two-way communication. Express appreciation for his insights and discussions.

**Note:**
Ensure each question is open-ended to allow Noah to provide insights based on his experiences and perspectives. The goal is to not only assess his technical and leadership competencies but also to understand his ability to integrate into the AI Fund's culture and contribute to its long-term goals.